In [28]:
from pymongo import MongoClient
from datetime import date

In [2]:
client = MongoClient()
db = client.bills
bill_info = db.bill_info
predictions = db.predictions
prev_predictions = db.prev_predictions


In [3]:
leg_id = 'H R 21'
cong_id = '116'

In [4]:
docs = bill_info.find({'leg_id': leg_id, 'congress_id': cong_id})

for doc in docs:
    print(doc['leg_id'])
    print('\tBill Status in database: {}'.format(doc['bill_status']))

H R 21
	Bill Status in database: Passed House


In [26]:
print('-------------------')
new = predictions.find({'leg_id': leg_id, 'congress_id': cong_id})
for n in new:
    print('{} Current Predictions'.format(n['leg_id']))
    print('\tDate: {}'.format(n['pred_date']))
    print('\t\tNLP Prediction:\t\t{}'.format(n['nlp_pred_proba']))
    print('\t\tNumerical Prediction:\t{}'.format(n['num_pred_proba']))
    print('\t\t\tCombined Prediction:\t{}'.format(n['pred_proba']))

-------------------
H R 21 Current Predictions
	Date: 01/11/2019
		NLP Prediction:		0.26712
		Numerical Prediction:	0.82674
			Combined Prediction:	0.54693


In [41]:
print('-------------------')
prev = prev_predictions.find({'leg_id': leg_id, 'congress_id': cong_id})
for n in prev:
    print('{} Previous Predictions'.format(n['leg_id']))
    if 'pred_date' in n.keys():
        print('\tDate: {}'.format(n['pred_date']))
    else: 
        print('\tDate: 01/08/2019')
        
    if 'nlp_pred_proba' in n.keys():
        print('\t\tNLP Prediction:\t\t{}'.format(n['nlp_pred_proba']))
    if 'num_pred_proba' in n.keys():
        print('\t\tNumerical Prediction:\t{}'.format(n['num_pred_proba']))
    if 'pred_proba' in n.keys():
        print('\t\t\tCombined Prediction:\t{}'.format(n['pred_proba']))
    if 'probability' in n.keys():
        print('\t\t\tOld Prediction:\t\t{}'.format(n['probability']))


-------------------
H R 21 Previous Predictions
	Date: 01/10/2019
		NLP Prediction:		0.26712
		Numerical Prediction:	0.82674
			Combined Prediction:	0.54693
H R 21 Previous Predictions
	Date: 01/08/2019
			Old Prediction:		0.26712


In [38]:
date.today().isoformat()

'2019-01-11'